In [23]:
import httpx
from api.src.schemas.authorities.authority import Uri, Variant, AdminMetadata, Element
from typing import Optional
from pydantic import BaseModel
from pysolr import Solr

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">166</int>\n</lst>\n</response>\n'

In [10]:
class Subject(BaseModel):
    type: str 
    adminMetadata: AdminMetadata 
    elementList: list[Element]
    note: Optional[str]
    hasReciprocalAuthority: Optional[list[Uri]]
    hasReciprocalExternalAuthority: Optional[list[Uri]]
    hasBroaderAuthority: Optional[list[Uri]]
    hasBroaderExternalAuthority: Optional[list[Uri]]
    hasNarrowerAuthority: Optional[list[Uri]]
    hasNarrowerExternalAuthority: Optional[list[Uri]]
    hasCloseExternalAuthority: Optional[list[Uri]]
    hasExactExternalAuthority: Optional[list[Uri]]
    hasVariant: Optional[list[Variant]]
    subjectOf: Optional[list[Uri]]
    isMemberOfMADSCollection: str

In [39]:
authority = 'http://id.loc.gov/authorities/subjects/sh85095726'
url = f'http://localhost:8000/authorities/import/loc/subject?uri={authority}'
r = httpx.get(url, timeout=20)
r.json()

{'type': 'Topic',
 'adminMetadata': {'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
  'descriptionModifier': 'http://id.loc.gov/vocabulary/organizations/brmninpa',
  'creationDate': '2023-05-18',
  'descriptionLanguage': 'http://id.loc.gov/vocabulary/languages/por',
  'generationProcess': 'BiblioKeia v.1',
  'generationDate': '2023-05-18T15:20:28',
  'identifiedBy': [{'type': 'Lccn',
    'assigner': 'http://id.loc.gov/vocabulary/organizations/dlc',
    'value': 'sh85095726'}],
  'status': {'value': 'mstatus:new', 'label': 'novo'}},
 'elementList': [{'type': 'TopicElement',
   'elementValue': {'value': 'Ornitologia', 'lang': 'pt'}}],
 'note': None,
 'hasReciprocalAuthority': None,
 'hasReciprocalExternalAuthority': [{'value': 'http://id.loc.gov/authorities/subjects/sh85014310',
   'label': {'value': 'Birds', 'lang': 'en'},
   'base': 'loc'}],
 'hasBroaderAuthority': None,
 'hasBroaderExternalAuthority': [{'value': 'http://id.loc.gov/authorities/subjects/sh85149983',
   'l

In [40]:
request = Subject(**r.json())
id = 'bkau-1'

In [18]:
def MakeLabel(elementList):
    labels = [i.elementValue.value for i in elementList]
    label = ", ".join(labels)
    
    return label

In [41]:
doc = {
        'id': id,
        'type': request.type,
        "creationDate": request.adminMetadata.creationDate.strftime('%Y-%m-%d'), 
        "label": f'{MakeLabel(request.elementList)}' ,
        "isMemberOfMADSCollection": request.isMemberOfMADSCollection
        }
doc

{'id': 'bkau-1',
 'type': 'Topic',
 'creationDate': '2023-05-18',
 'label': 'Ornitologia',
 'isMemberOfMADSCollection': 'http://bibliokeia.com/authorities/Topic/'}

In [58]:
if request.hasVariant:
    variants = list()
    for i in request.hasVariant:
        label = [j.elementValue.value for j in i.elementList]
        label = "--".join(label)
        variants.append(label)
    doc['variant'] = variants
    print(variants)

['Biologia aviária']


In [49]:
for k, v in request:

        if v and k not in ['type', 'adminMetadata', 'elementList', 'isMemberOfMADSCollection', 'hasVariant']:
                pass
                # for i in v:
                #         uri = {
                #                 'id': f"{id}/{k}#{i.value.split('/')[-1]}",
                #                 'uri': i.value, 
                #                 'label': i.label.value, 
                #                 'lang': i.label.lang}
        elif k == 'hasVariant' and v:
                for i in v:
                         
                        print(variant)


                # uri = [{'id': i.value.split("/")[-1], 'uri': i.value, 'label': i.label.value, 'lang': i.label.lang} for i in v]
                # doc[f'{k}'] = uri


type='Topic' elementList=[Element(type='TopicElement', elementValue=Label(value='Biologia aviária', lang='pt'))] componentList=None


In [43]:
request.hasVariant

[Variant(type='Topic', elementList=[Element(type='TopicElement', elementValue=Label(value='Biologia aviária', lang='pt'))], componentList=None)]

In [24]:
solr = Solr('http://localhost:8983/solr/authorities/', timeout=10)

doc = {
 "id":"bkau-1",
 "hasBroaderAuthority":{ "set": { 
     'uri': 'http://id.loc.gov/authorities/subjects/sh85149983',
  'label': 'Zoology',
  'lang': 'en'}}
}

responseSolr = solr.add([doc], commit=True)

In [62]:
solr.delete(q=f"id:bkau-2/hasReciprocalExternalAuthority#sh85048830",  commit=True)

'<?xml version="1.0" encoding="UTF-8"?>\n<response>\n\n<lst name="responseHeader">\n  <int name="status">0</int>\n  <int name="QTime">72</int>\n</lst>\n</response>\n'

In [ ]:
{
  "authority": "https://bibliokeia.com/authorities/Topic/bkau-1",
  "uri": "http://id.loc.gov/authorities/subjects/sh85006228",
  "value": "string",
  "lang": "string",
  "type": "hasBroaderExternalAuthority"
}

In [ ]:
auId = request.authority.split("/")[-1]
uriId = request.uri.split("/")[-1]
